In [2]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from pypdf import PdfReader


In [3]:

reader = PdfReader("Options_Volatility_Script_Documentation.pdf")
text = ''

for page in reader.pages:
    text += page.extract_text()

print(text[:500])

Options Volatility Script: 
Documentation
Key Features:
Option Chain Data: Fetches live call/put options from NSE.
Historical Data: Retrieves daily OHLC data from NSE (last 90 days).
Historical Volatility HV Calculates past stock price fluctuations.
GARCH Volatility: Models future volatility based on past returns.
Implied Volatility IV Derives market's expectation of future volatility from 
option prices.
Excel Output: Saves all data into an organized Excel file.
Mathematical Formulas :
1.


In [ ]:
raw_text = text

text_splitter = CharacterTextSplitter(chunk_size = 1000, chunk_overlap = 100)